In [3]:
%run -i writefile2.py

In [4]:
%%writefile2
from tqdm import tqdm
import numpy as np
import sys

def predict_replace( df, ensemble, verbose = 1 ):

    # for predicted time range
    
    min_start = max([ l.length_l + l.lag for l in ensemble ])
    min_length = max([ l.length_r for l in ensemble ])
    
    start = min_start
    length = df.shape[ 0 ] - start
    
    assert start >= min_start
    assert length >= min_length
    assert start + length <= df.shape[ 0 ]
    
    lhs_chunks = [ ]
    rhs_chunks = [ ]
    weight_chunks = [ ]
    
    for learner in tqdm( ensemble, file = sys.stdout ):
    
        n_predictions = 1 + length - learner.length_r
        M = learner.linear_operator
        n_rows_total = M.shape[ 0 ] * n_predictions
        lag = learner.lag
    
        for i in range( n_predictions ):
    
            chunk = np.zeros(( M.shape[ 0 ], length ))
            chunk[ :, i : i + M.shape[ 1 ]] = M
            lhs_chunks.append( chunk )
    
            window = df.iloc[ start - lag - learner.length_l + i: start - lag + i, : ].to_numpy( )
            y = learner.predict( window )
            rhs_chunks.append( y )
    
            # each weak learner is now regarded equally important, independent of total rows occupied
            # otherwise learners with smaller windows and larger operators are favoured
            weight_chunks.append( np.ones( M.shape[ 0 ], ) * ( learner.weight / n_rows_total ))

    lhs = np.concatenate( lhs_chunks, axis = 0 )
    rhs = np.concatenate( rhs_chunks, axis = 0 )
    weight = np.concatenate( weight_chunks, axis = 0 )
    
    lhs = np.diag( weight ) @ lhs
    rhs = np.diag( weight ) @ rhs
    
    assert lhs.shape == ( rhs.shape[ 0 ], length )

    if verbose >= 1:
    
        print( f"lhs shape = { lhs.shape }" )
        print( f"lhs rank = { np.linalg.matrix_rank( lhs )}" )
        print( f"degrees of freedom = { length }" )

    prediction, *_ = np.linalg.lstsq( lhs, rhs, rcond = None )

    df_pred = df.copy( )
    df_pred.iloc[ :, : ] = np.nan
    df_pred.iloc[ start : start + length, :3 ] = prediction

    return df_pred